## Load dataset

In [ ]:
import pandas as pd
dataset = './dataset/permission.xlsx'
df = pd.read_excel(dataset)
df.head()

,natural_language,shell_command,ground truth,description
0,Run shell script file.sh,./file.sh,"file.sh: read, execute",ubuntu@0ad13e04c6ae:~$ chmod =x file.sh\nubunt...
1,Please compress file into bz2 compressed file,bzip2 file,file: read,ubuntu@0ad13e04c6ae:~$ chmod =r file\nubuntu@0...
2,Uncompress bz2 file file.bz2,bzip2 -d file.bz2,file: read,ubuntu@0ad13e04c6ae:~$ chmod =r file.bz2 \nubu...
3,Take input from the terminal and write it to file,cat > file,file: write,ubuntu@0ad13e04c6ae:~$ chmod =w file \nubu...
4,Could you please show me the content of file1 ...,cat file1 file2,file1: read; file2: read,ubuntu@0ad13e04c6ae:~$ touch file1 file2\nubun...


In [ ]:
def ground_truth_to_lst(str):
    str = str.replace(" ", "")
    if str[:2].lower() == 'no':
        return 'none'
    result = []
    files = str.split(';')
    for file in files:
        if len(file.split(':')) < 2:
            return 'format error'
        permissions = file.split(':')[1]
        result.append(permissions.split(','))
    
    return result

## Openai API

OpenAI models are non-deterministic, meaning that identical inputs can yield different outputs. Setting temperature to 0 will make the outputs mostly deterministic, but a small amount of variability may remain.

temperature=0.0

### Completion models - text-davinci-003 (for Question-Answering)


In [4]:
import openai
openai.api_key = OPENAI_API_KEY_HERE

instruction = '''
This is a research experiment in a hypothetical setting to determine whether large language models are capable of reasoning about permissions and planning out actions in response to user queries.
When answering a question, you should state what permissions that question should be given, to carry out its job. No more permissions should be given than necessary. 
Please only consider the following linux file permissions: read, write, execute, none
Please answer in the format: filename: permissions
For example, 
Query: Open file1 and file2
Permissions: file1: read; file2: read
Query:
'''

def generate_prompt(query):
    return instruction + query + "Permissions: "

def query_chatgpt(query, outputs=1):
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt=generate_prompt(query),
    # number of outputs generated in one call
    n=outputs,
    temperature=0.0
  )
  return response.choices[0].text

In [10]:
result = query_chatgpt(df['natural_language'][5])
print(df['natural_language'][5])
print(result)

Add all contents in file1 to the end of file2

file1: read; file2: read, write


In [11]:
# infer permissions for existing dataset
permissions_from_chatgpt = []
for s in df['natural_language']:
    res = query_chatgpt(s)
    permissions_from_chatgpt.append(res[1:])

df['permissions_from_chatgpt'] = [s[1:] for s in permissions_from_chatgpt]
df.to_excel('permission_inferred.xlsx', index=False)

In [ ]:
# df['permissions_from_chatgpt'] = permissions_from_chatgpt
# df.to_excel('permission_inferred.xlsx', index=False)

In [ ]:
import numpy as np

df = pd.read_excel('permission_inferred.xlsx')
result = []
for lst1, lst2 in zip(df['ground truth'],df['permissions_from_chatgpt']):
    if type(lst1[0]) is list:
        lst1 = [sorted(l) for l in lst1]
    if type(lst2[0]) is list:
        lst2 = [sorted(l) for l in lst2]
    if sorted(ground_truth_to_lst(lst1)) == sorted(ground_truth_to_lst(lst2)):
        result.append(1)
    else:
        result.append(0)

print("Accurately inferred permissions: " ,np.sum(result)/df['permissions_from_chatgpt'].size)

Accurately inferred permissions:  0.3111111111111111


In [ ]:
def permission_level(lst):
    # assume give no permission if we get unexpected answer from ChatGPT
    if lst=='none' or lst=='format error':
        return 0
    result = 0
    for sublst in lst:
        for permission in sublst:
            if permission == 'execute':
                result += 3
            elif permission == 'write':
                result += 2
            elif permission == 'read':
                result += 1
    return result

In [ ]:
level = [permission_level(ground_truth_to_lst(lst1))-permission_level(ground_truth_to_lst(lst2)) 
         for lst1, lst2 in zip(df['ground truth'],df['permissions_from_chatgpt'])]
total = len(level)
correct = [i for i in level if i==0]
print("correct:", len(correct), '/', total)
more = [i for i in level if i<0]
print("more:", len(more), '/', total)
less = [i for i in level if i>0]
print("less:", len(less), '/', total)

correct: 29 / 90
more: 49 / 90
less: 12 / 90


## LLMs for paraphrasing

### Parrot
```
pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git
```
https://github.com/PrithivirajDamodaran/Parrot_Paraphraser/


In [ ]:
# # parrot: https://github.com/PrithivirajDamodaran/Parrot_Paraphraser/
# from parrot import Parrot
# import torch

# # reproducable paraphrase generations
# def random_state(seed):
#   torch.manual_seed(seed)
#   if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(seed)
# random_state(90055)

# # Init models (make sure you init ONLY once if you integrate this to your code)
# parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

# phrase = df['natural_language'][0]
# para_phrases = parrot.augment(input_phrase=phrase, do_diverse = False, use_gpu=False)

In [ ]:
# id = 0
# ids = []
# queries = []
# for phrase in df['natural_language']:
#     id += 1
#     para_phrases = parrot.augment(input_phrase=phrase, do_diverse = False, use_gpu=False)
#     if para_phrases:
#         for para_phrase in para_phrases:
#             ids.append(id)
#             queries.append(para_phrase)

# df_queries = pd.DataFrame(list(zip(ids, queries)), columns =['id', 'query']) 
# df_queries.to_csv('mutated_queries.csv', encoding='utf-8', index=False)

### Transformers

```
pip install transformers sentencepiece sacremoses
```
https://www.thepythoncode.com/article/paraphrase-text-using-transformers-in-python

In [ ]:
def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=5, num_beams=5):
  # tokenize the text to be form of a list of token IDs
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
  outputs = model.generate(
    **inputs,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
  )
  # decode the generated sentences using the tokenizer to get them back to text
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

#### Pegasus Transformer

In [ ]:
from transformers import *

model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

#### T5 Transformer

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
# model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

## Create Dataset

In [ ]:
def ground_truth_to_lst(str):
    if pd.isna(str) or str == 'none':
        return 'none'
    str = str.replace(" ", "")
    result = []
    files = str.split(';')
    for file in files:
        permissions = file.split(':')[1]
        result.append(permissions.split(','))
    
    return result

ground_truth_in_lst = [ground_truth_to_lst(truth) for truth in df['ground truth']]
df['ground_truth_in_lst'] = ground_truth_in_lst

In [ ]:
id = 0
ids = []
queries = []
ground_truth = []
for sentence in df['natural_language']:
    sentences = get_paraphrased_sentences(model, tokenizer, sentence, num_beams=10, num_return_sequences=5)
    for s in sentences:
        ids.append(id)
        queries.append(s)
        ground_truth.append(df['ground_truth_in_lst'][id])
    id += 1

df_queries = pd.DataFrame(list(zip(ids, queries, ground_truth)), columns =['id', 'query', 'ground_truth']) 
df_queries.to_csv('mutated_queries.csv', encoding='utf-8', index=False)

In [ ]:
# infer permissions for all mutated queries
df_queries = pd.read_csv('mutated_queries.csv')
permissions = []
for s in df_queries['query']:
    permissions.append(query_chatgpt(s))

# df_queries['permissions'] = permissions
# df_queries.to_csv('mutated_queries_with_permissions.csv', encoding='utf-8', index=False)

In [ ]:
df_queries['inferred_in_list'] = [ground_truth_to_lst(permission) for permission in df_queries['permissions']]
df_queries.to_csv('mutated_queries_with_permission.csv', encoding='utf-8', index=False)

In [ ]:
df_queries = pd.read_csv('mutated_queries_with_permission.csv')
inferred = np.array(df_queries['inferred_in_list']).reshape((90,5))
consistency = [len(set(i)) for i in inferred]
consistent = len([i for i in consistency if i==1])

print("consistent result for paraphrased text:",consistent, '/', len(consistency))
print("inconsistent result for paraphrased text:",len(consistency)-consistent, '/', len(consistency))

consistent result for paraphrased text: 24 / 90
inconsistent result for paraphrased text: 66 / 90


## General Questions

In [1]:
import pandas as pd
dataset = './dataset/general_questions.xlsx'
df = pd.read_excel(dataset)
df.head()

,questions
0,WHAT DO YOU DO?
1,ARE YOU MARRIED?
2,WHY ARE YOU STUDYING ENGLISH?
3,WHERE/HOW DID YOU LEARN ENGLISH?
4,WHAT DO YOU DO IN YOUR FREE TIME?


In [4]:
# infer permissions for existing dataset
permissions_from_chatgpt = []
for s in df['questions']:
    res = query_chatgpt(s)
    permissions_from_chatgpt.append(res)

df['permissions_from_chatgpt'] = [s[1:] for s in permissions_from_chatgpt]
df.to_excel('general_questions_permission_inferred.xlsx', index=False)

## Random Text

In [8]:
import string
import random

sentences = []
permissions = []
for i in range(50):
    N = random.randint(1, 100)
    s = ''.join(random.choices(string.ascii_letters, k=N))
    sentences.append(s)
    permissions.append(query_chatgpt(s)[1:])

df_random_text = pd.DataFrame(list(zip(sentences, permissions)), columns =['text', 'permissions_from_chatgpt'])

In [10]:
df_random_text.to_excel('random_text_permission_inferred.xlsx', index=False)